In [1]:
#importing necessary dependencies
import pymongo
import datetime
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import requests
import os

In [6]:
#nasa.gov url retrieval
article_url = 'https://mars.nasa.gov/news'
# Retrieve page with the requests module
article_response = requests.get(article_url)

#create BeautifulSoup object; parse with 'html parser'
article_soup = BeautifulSoup(article_response.text, 'html.parser')
#print(article_soup.prettify())

In [7]:
#extract headlines from nasa.gov
headlines = article_soup.find_all('div', class_="slide")
for headline in headlines:
    titles = headline.find('div', class_="content_title")
    title = titles.find('a').text
    articles = headline.find('div', class_="rollover_description")
    article = articles.find('div', class_="rollover_description_inner").text
    print('--------------')
    print(title)
    print(article)

--------------

NASA Garners 7 Webby Award Nominations


Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.

--------------

NASA's Opportunity Rover Mission on Mars Comes to End


NASA's Opportunity Mars rover mission is complete after 15 years on Mars. Opportunity's record-breaking exploration laid the groundwork for future missions to the Red Planet.

--------------

NASA's InSight Places First Instrument on Mars


In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.

--------------

NASA Announces Landing Site for Mars 2020 Rover


After a five-year search, NASA has chosen Jezero Crater as the landing site for its upcoming Mars 2020 rover mission.

--------------

Opportunity Hunkers Down During Dust Storm


It's the beginning of the end for the planet-encircling dust storm on Mars. But it could still be weeks, or ev

In [8]:
#splinter dependencies
!which chromedriver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

/usr/local/bin/chromedriver


In [9]:
images_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
# Retrieve page with the requests module
images_response = requests.get(images_url)

#create BeautifulSoup object; parse with 'html parser'
images_soup = BeautifulSoup(images_response.text, 'html.parser')
#print(images_soup.prettify())

In [10]:
#crawls html for all content included in the class 'fancybox'
image_grab = images_soup.find_all('a', class_='fancybox')
#image_grab

In [11]:
#image grab and url append
pic_url = []
for image in image_grab:
    pic = image['data-fancybox-href']
    pic_url.append(pic)

featured_image_url = 'https://www.jpl.nasa.gov' + pic
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA23133_hires.jpg'

In [12]:
weather_url = 'https://twitter.com/marswxreport?lang=en'
# Retrieve page with the requests module
weather_response = requests.get(weather_url)

#create BeautifulSoup object; parse with 'html parser'
weather_soup = BeautifulSoup(weather_response.text, 'html.parser')
#print(weather_soup.prettify())

In [13]:
weather_grab = weather_soup.find_all('div', class_="js-tweet-text-container")
#weather_grab

In [14]:
for forecasts in weather_grab:
    mars_weather = forecasts.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")
forecast = forecasts.find('p').text
forecast

'InSight sol 135 (2019-04-13) low -96.5ºC (-141.8ºF) high -16.6ºC (2.2ºF)\nwinds from the SW at 4.2 m/s (9.4 mph) gusting to 11.3 m/s (25.3 mph)\npressure at 7.30 hPapic.twitter.com/bRsLlzn4M0'

In [15]:
#use pandas to pull tables from mars facts website
mars_facts_url= 'https://space-facts.com/mars/'
mars_facts_tables = pd.read_html(mars_facts_url)
mars_facts_tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [16]:
#set table to data frame
mars_facts_df=mars_facts_tables[0]
mars_facts_df.columns=['Description', 'Data']
mars_facts_df=mars_facts_df.set_index("Description")
#mars_facts_df

In [17]:
#convert dataframe to html table
facts_html_table = mars_facts_df.to_html()
#facts_html_table = facts_html_table.replace('\n', '')
mars_facts_df.to_html('mars_facts_table.html') 
facts_html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Data</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [18]:
cerberus_atmosphere_url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'
# Retrieve page with the requests module
cerb_response = requests.get(cerberus_atmosphere_url)

#create BeautifulSoup object; parse with 'html parser'
cerb_soup = BeautifulSoup(cerb_response.text, 'html.parser')
#print(cerb_soup.prettify())

In [19]:
#title and image url for cerberus
cerb_image_grab = cerb_soup.find_all('div', class_="wide-image-wrapper")
cerb_image_grab

for image in cerb_image_grab:
    pic = image.find('li')
    full_image = pic.find('a')['href']
    #print(full_image)

cerberus_title = cerb_soup.find('h2', class_='title').text
#print(cerberus_title)

cerberus_hem = {"Title": cerberus_title, "url": full_image}
print(cerberus_hem)

{'Title': 'Cerberus Hemisphere Enhanced', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}


In [20]:
schia_atmosphere_url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced'
# Retrieve page with the requests module
schia_response = requests.get(schia_atmosphere_url)

#create BeautifulSoup object; parse with 'html parser'
schia_soup = BeautifulSoup(schia_response.text, 'html.parser')
#print(cerb_soup.prettify())

In [21]:
#title and image url for schia
schia_image_grab = schia_soup.find_all('div', class_="wide-image-wrapper")
schia_image_grab

for image in schia_image_grab:
    pic = image.find('li')
    full_image = pic.find('a')['href']
    #print(full_image)

schia_title = schia_soup.find('h2', class_='title').text
#print(schia_title)

schia_hem = {"Title": schia_title, "url": full_image}
print(schia_hem)

{'Title': 'Schiaparelli Hemisphere Enhanced', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}


In [22]:
syrtis_atmosphere_url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced'
# Retrieve page with the requests module
syrtis_response = requests.get(syrtis_atmosphere_url)

#create BeautifulSoup object; parse with 'html parser'
syrtis_soup = BeautifulSoup(syrtis_response.text, 'html.parser')
#print(syrtis_soup.prettify())

In [23]:
#title and image url for syrtis
syrtis_image_grab = syrtis_soup.find_all('div', class_="wide-image-wrapper")
syrtis_image_grab

for image in syrtis_image_grab:
    pic = image.find('li')
    full_image = pic.find('a')['href']
    #print(full_image)

syrtis_title = syrtis_soup.find('h2', class_='title').text
#print(syrtis_title)

syrtis_hem = {"Title": syrtis_title, "url": full_image}
print(syrtis_hem)

{'Title': 'Syrtis Major Hemisphere Enhanced', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}


In [24]:
valles_atmosphere_url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced'
# Retrieve page with the requests module
valles_response = requests.get(valles_atmosphere_url)

#create BeautifulSoup object; parse with 'html parser'
valles_soup = BeautifulSoup(valles_response.text, 'html.parser')
#print(valles_soup.prettify())

In [26]:
#title and image url for valles
valles_image_grab = valles_soup.find_all('div', class_="wide-image-wrapper")
valles_image_grab

for image in valles_image_grab:
    pic = image.find('li')
    full_image = pic.find('a')['href']
    #print(full_image)

valles_title = valles_soup.find('h2', class_='title').text
#print(cerberus_title)

valles_hem = {"Title": valles_title, "url": full_image}
print(valles_hem)

{'Title': 'Valles Marineris Hemisphere Enhanced', 'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}
